In [20]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils


In [21]:
# Устанавливаем seed для повторяемости результатов
np.random.seed(42)

TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
OUTPUT_FILE = "submission.csv"
num_classes = 10
# Размер изображения
img_rows, img_cols = 28, 28

In [22]:
# Загружаем данные
train_dataset = np.loadtxt(TRAIN_FILE, skiprows=1, dtype='int', delimiter=",")
# Выделяем данные для обучения
x_train = train_dataset[:, 1:]
# Переформатируем данные в 2D, бэкенд Tensorflow
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
# Нормализуем данные
x_train = x_train.astype("float32")
x_train /= 255.0
# Выделяем правильные ответы
y_train = train_dataset[:, 0]
# Преобразуем правильные ответы в категоризированное представление
y_train = np_utils.to_categorical(y_train)

In [23]:
#Загружаем данные для предсказания
test_dataset = np.loadtxt(TEST_FILE, skiprows=1, delimiter=",")
# Переформатируем данные в 2D, бэкенд TensorFlow
x_test = test_dataset.reshape(test_dataset.shape[0], img_rows, img_cols, 1)
# Нормализуем данные
x_test = x_test.astype("float32")
x_test /= 255.0


In [24]:
# Создаем модель


model = Sequential()

# Свёрточный слой
# включает в себя для каждого канала свой фильтр, (ядро свёртки) который обрабатывает предыдущий слой по фрагментам 
# фильтр свёртки, похож на фонарик, которым можно светить в темном помещении, пытаясь найти нужные признаки
# Весовые коэффициенты ядра свёртки (небольшой матрицы) неизвестны и устанавливаются в процессе обучения.

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#еще один такой же слой
model.add(Conv2D(64, (3, 3), activation='relu'))

# слой пулинга (иначе подвыборки, субдискретизации) представляет собой нелинейное уплотнение карты признаков
# eсли на предыдущей операции свёртки уже были выявлены некоторые признаки, то для дальнейшей обработки 
# настолько подробное изображение уже не нужно и оно уплотняется до менее подробного. 

model.add(MaxPooling2D(pool_size=(2, 2)))

#отключаем рандомно N количество нейронов, для предотвращения переобучения
model.add(Dropout(0.25))

#Flatten слой преобразует двумерные вектора признаков, в одномерный вектор. 
#Полносвязные слои, используемые для классификации, не могут работать с двумерными данными на входе, им нужен одномерный вектор
model.add(Flatten())
#Полносвязный слой
#Полносвязная часть реализует классификацию - определяет, 
#что за объект находится на изображении на основе признаков, которые извлекла сверточная часть
model.add(Dense(128, activation='relu'))
#еще один DropOut чтобы не было переобучения 
model.add(Dropout(0.5))
#Выходной слой
model.add(Dense(num_classes, activation='softmax'))
# Компилируем модель
# Adadelta - популярная форма градиентного спуска, смысл которого в нахождении глобального минимума функции ошибки 
#(желаемое - полученное), которая зависит от весов
# Если по простому, то с помощью градиентного спуска мы коректируем веса нашей сети, чем ближе к глобальному минимуму тем лучше результаты.
# Градиентный спуск похож на альпиниста, который спускается по шагу с горы в полной темноте
# В такой ситуации очень легко попасть в локальный минимум или пропустить глобальный минимум и пойти не в том направлении
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
__________

In [25]:
# Обучаем сеть
model.fit(x_train, y_train, batch_size=200, epochs=10, verbose=2)

# Предсказываем значения
predictions = model.predict(x_test)
# Преобразуем метки в классы
predictions = np.argmax(predictions, axis=1)



Epoch 1/10
 - 114s - loss: 0.5044 - acc: 0.8462
Epoch 2/10
 - 116s - loss: 0.1652 - acc: 0.9511
Epoch 3/10
 - 108s - loss: 0.1168 - acc: 0.9657
Epoch 4/10
 - 108s - loss: 0.0929 - acc: 0.9724
Epoch 5/10
 - 114s - loss: 0.0806 - acc: 0.9757
Epoch 6/10
 - 109s - loss: 0.0705 - acc: 0.9788
Epoch 7/10
 - 106s - loss: 0.0651 - acc: 0.9805
Epoch 8/10
 - 101s - loss: 0.0581 - acc: 0.9825
Epoch 9/10
 - 104s - loss: 0.0548 - acc: 0.9827
Epoch 10/10
 - 107s - loss: 0.0491 - acc: 0.9852


In [26]:
# Запись в файл
out = np.column_stack((range(1, predictions.shape[0]+1), predictions))
np.savetxt(OUTPUT_FILE, out, header="ImageId,Label", comments="", fmt="%d,%d")